# OpenAI Text Embedding API and Google Cloud Vertex AI Matching Engine
For content similarity analysis (text demonstrated)

## Installation

Install the latest version of Cloud Storage, BigQuery and Vertex AI SDKs for Python.

In [ ]:
# Install the packages
! pip3 install --upgrade pip
! pip3 install --upgrade \
    google-cloud-aiplatform \
    google-cloud-storage \
    grpcio-tools \
    openai \
    transformers

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Before you begin
### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "{PROJECT ID}"

# Set the project id
!gcloud config set project {PROJECT_ID} --quiet

### Region Selection

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"

### OpenAI Token

In [ ]:
%env OPENAI_API_KEY {TOKEN HERE}

### VPC Network

In [ ]:
VPC_NETWORK = "{VPC}"
PEERING_RANGE_NAME = "{RANGE}"

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "gs://{BUCKET NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Next, put your data in the bucket

## Prepare the data

In [ ]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 4
# Directory to store the text data locally
DATA_DIR = 'data/'
# File type wildcard
FILE_TYPE = '*.md'

In [ ]:
!mkdir {DATA_DIR}
!gsutil -m cp "$BUCKET_URI/$FILE_TYPE" "$DATA_DIR"

### Read the data into memory.

In [ ]:
import pandas as pd
import glob
import openai
import os

from transformers import GPT2Tokenizer

# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
# Using a reasonable/similar open-source tokenizer, to avoid tiktoken (requires Python 3.8+)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
MAX_TOKENS = 8191

# Function to read the content of a file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def count_tokens(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    token_count = len(input_ids)
    return token_count

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

openai.api_key = os.getenv("OPENAI_API_KEY")

# Fetch all files of the specified type in the directory
file_paths = glob.glob(os.path.join(DATA_DIR, FILE_TYPE))

# Read the content of each file as a string and store in a list
file_contents = [read_file(file_path) for file_path in file_paths]

# Create a Pandas DataFrame with the file contents
df = pd.DataFrame(file_contents, columns=['content'])

# Add a column with the file names
file_names = [os.path.basename(file_path) for file_path in file_paths]
df['file_name'] = file_names

df['embedding'] = df['content'].apply(
    lambda x: get_embedding(x, model='text-embedding-ada-002') if count_tokens(x) <= MAX_TOKENS else None
)

In [ ]:
! mkdir output
df = df.dropna(subset=['embedding'])
df.to_csv('output/embedded_content.csv', index=False)

#### Save the data in JSONL format.

The data must be formatted in JSONL format, which means each embedding dictionary is written as a JSON string on its own line.

Additionally, to demonstrate the filtering functionality, the `restricts` key is set such that each embedding has a different `class`, `even` or `odd`. These are used during the later matching step to filter for results.
See additional information of filtering here: https://cloud.google.com/vertex-ai/docs/matching-engine/filtering

In [ ]:
import json

# Apply transformations to the DataFrame
df["id"] = df.index
df["id"] = df["id"].astype('str')

# Write the DataFrame to the file in JSON format
with open("posts.json", "w") as f:
    for _, row in df.iterrows():
        json_row = {"id": row["id"], "embedding": row["embedding"]}
        json_line = json.dumps(json_row) + "\n"
        f.write(json_line)

Upload the training data to GCS.

In [ ]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/matching_engine/initial/"
! gsutil cp posts.json {EMBEDDINGS_INITIAL_URI}

## Create Indexes


### Create Brute Force Index (for Ground Truth)

The brute force index uses a naive brute force method to find the nearest neighbors. This method is not fast or efficient. Hence brute force indices are not recommended for production usage. They are to be used to find the "ground truth" set of neighbors, so that the "ground truth" set can be used to measure recall of the indices being tuned for production usage. To ensure an apples to apples comparison, the `distanceMeasureType` and `dimensions` of the brute force index should match those of the production indices being tuned.

Create the brute force index configuration:

In [ ]:
from google.cloud import aiplatform

df['embedding_length'] = df['embedding'].apply(len)

assert df['embedding_length'].nunique() == 1, "All embedding_length values are not the same."

brute_force_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name="POSTS",
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    distance_measure_type="COSINE_DISTANCE",
    dimensions=int(df['embedding_length'].unique()[0]),
    description="Posts index (brute force)",
)

In [ ]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.resource_name
INDEX_BRUTE_FORCE_RESOURCE_NAME

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex(
    index_name=INDEX_BRUTE_FORCE_RESOURCE_NAME
)

## Create an IndexEndpoint with VPC Network

In [ ]:
# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="Posts similarity scoring",
    network=VPC_NETWORK_FULL,
)

In [ ]:
INDEX_ENDPOINT_NAME = my_index_endpoint.resource_name
INDEX_ENDPOINT_NAME

## Deploy Indexes

### Deploy Brute Force Index

In [ ]:
DEPLOYED_BRUTE_FORCE_INDEX_ID = "posts_brute_force_deployed"

In [ ]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=brute_force_index, deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID
)

In [ ]:
from google.cloud import aiplatform

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name = ""
)

my_index_endpoint.deployed_indexes

## Create Online Queries

After you built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example 'us-central1' in this tutorial).

In [ ]:
NUM_NEIGHBOURS = 4
NUM_NEIGHBOURS

In [ ]:
import socket

def is_port_open(host, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.settimeout(3)  # Set a timeout (in seconds) for the connection attempt
            sock.connect((host, port))
            return True
        except socket.error:
            return False

# Usage example
host = my_index_endpoint.deployed_indexes[0].private_endpoints.match_grpc_address
port = 10000

if is_port_open(host, port):
    print(f"Port {port} is open on {host}")
else:
    print(f"Port {port} is not open on {host}")

In [ ]:
# Test query
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import \
    Namespace

# Test query
responses = my_index_endpoint.match(
    deployed_index_id="posts_brute_force_deployed",
    queries=[list(df.iloc[1]['embedding'])],
    num_neighbors=4
)

In [ ]:
print(df.iloc[1]['file_name'])

for response in responses:
    for neighbor in response:
        print(neighbor)
        print(df["file_name"].iloc[int(neighbor.id) - 1])

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)
brute_force_index.delete()